# Train SALLY ensemble

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge, EnsembleForge

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'

## Settings

In [4]:
n_estimators = 10

## Training function

In [5]:
def train_ensemble(filename, use_tight_cuts=True, n_estimators=10, **kwargs):
    cut_label = '_tight' if use_tight_cuts else ''
    
    ensemble = EnsembleForge(n_estimators, debug=False)

    ensemble.train_all(
        method='sally',
        x_filename=[sample_dir + 'train_local{}/x_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        t_xz0_filename=[sample_dir + 'train_local{}/t_xz_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        **kwargs
    )

    ensemble.calculate_expectation(
        x_filename=sample_dir + 'validation{}/x_validation.npy'.format(cut_label)
    )

    ensemble.save(model_dir + 'sally_ensemble_' + filename)

## All observables

In [ ]:
train_ensemble('all', use_tight_cuts=False)

07:36  Training 10 estimators in ensemble
07:36  Training estimator 1 / 10 in ensemble
07:36  Starting training
07:36    Method:                 sally
07:36    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/x_train_0.npy
07:36                   t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/t_xz_train_0.npy
07:36    Features:               all
07:36    Method:                 sally
07:36    Hidden layers:          (100, 100)
07:36    Activation function:    tanh
07:36    Batch size:             128
07:36    Trainer:                amsgrad
07:36    Epochs:                 50
07:36    Learning rate:          0.001 initially, decaying to 0.0001
07:36    Validation split:       0.25
07:36    Early stopping:         True
07:36    Scale inputs:           True
07:36    Regularization:         None
07:36  Loading training data
07:36  Found 1000000 samples with 2 paramet

07:45    Trainer:                amsgrad
07:45    Epochs:                 50
07:45    Learning rate:          0.001 initially, decaying to 0.0001
07:45    Validation split:       0.25
07:45    Early stopping:         True
07:45    Scale inputs:           True
07:45    Regularization:         None
07:45  Loading training data
07:45  Found 1000000 samples with 2 parameters and 27 observables
07:45  Rescaling inputs
07:45  Observable ranges:
07:45    x_1: mean -9.239897735824343e-17, std 1.0000000000000209, range -1.2733982205951953 ... 46.967201549377855
07:45    x_2: mean 3.892175470809889e-17, std 1.000000000000059, range -1.7275326350199396 ... 1.7301371229199798
07:45    x_3: mean -2.99455340524446e-14, std 1.0000000000000244, range -0.6581110421560781 ... 29.45231630664146
07:45    x_4: mean 3.6637715083998044e-16, std 1.0000000000000258, range -5.242219236491054 ... 4.710909019701929
07:45    x_5: mean 4.166267331129347e-17, std 0.999999999999968, range -0.8279265579300497 ... 39.8

07:57    Epoch 40: train loss 20.7867 (mse_score: 20.7867)
07:57              val. loss  15.0743 (mse_score: 15.0743)
07:58    Epoch 41: train loss 20.7619 (mse_score: 20.7619)
07:58              val. loss  15.0976 (mse_score: 15.0976)
07:58    Epoch 42: train loss 20.7211 (mse_score: 20.7211)
07:58              val. loss  15.0720 (mse_score: 15.0720)
07:58    Epoch 43: train loss 20.6935 (mse_score: 20.6935)
07:58              val. loss  15.0767 (mse_score: 15.0767)
07:59    Epoch 44: train loss 20.6629 (mse_score: 20.6629)
07:59              val. loss  15.0820 (mse_score: 15.0820)
07:59    Epoch 45: train loss 20.6299 (mse_score: 20.6299)
07:59              val. loss  15.0872 (mse_score: 15.0872)
07:59  No improvement for 20 epochs, stopping training
07:59  Early stopping after epoch 25, with loss 15.01 compared to final loss 15.09
07:59  Finished training
07:59  Training estimator 3 / 10 in ensemble
07:59  Starting training
07:59    Method:                 sally
07:59    Training da

08:08              val. loss  17.7618 (mse_score: 17.7618)
08:08    Epoch 28: train loss 22.2819 (mse_score: 22.2819)
08:08              val. loss  17.8142 (mse_score: 17.8142)
08:08    Epoch 29: train loss 22.2002 (mse_score: 22.2002)
08:08              val. loss  17.7843 (mse_score: 17.7843)
08:09    Epoch 30: train loss 22.1338 (mse_score: 22.1338)
08:09              val. loss  17.8190 (mse_score: 17.8190)
08:09    Epoch 31: train loss 22.0757 (mse_score: 22.0757)
08:09              val. loss  17.7585 (mse_score: 17.7585)
08:09    Epoch 32: train loss 22.0128 (mse_score: 22.0128)
08:09              val. loss  17.8426 (mse_score: 17.8426)
08:10    Epoch 33: train loss 21.9365 (mse_score: 21.9365)
08:10              val. loss  17.7959 (mse_score: 17.7959)
08:10    Epoch 34: train loss 21.8779 (mse_score: 21.8779)
08:10              val. loss  17.8329 (mse_score: 17.8329)
08:10    Epoch 35: train loss 21.8179 (mse_score: 21.8179)
08:10              val. loss  17.8689 (mse_score: 17.868

08:29    Epoch 24: train loss 34.8782 (mse_score: 34.8782)
08:29              val. loss  30.7689 (mse_score: 30.7689)
08:29    Epoch 25: train loss 34.8256 (mse_score: 34.8256)
08:29              val. loss  30.7974 (mse_score: 30.7974)
08:29    Epoch 26: train loss 34.7229 (mse_score: 34.7229)
08:29              val. loss  31.0003 (mse_score: 31.0003)
08:30    Epoch 27: train loss 34.6452 (mse_score: 34.6452)
08:30              val. loss  30.8496 (mse_score: 30.8496)
08:30    Epoch 28: train loss 34.5561 (mse_score: 34.5561)
08:30              val. loss  30.8253 (mse_score: 30.8253)
08:30    Epoch 29: train loss 34.4853 (mse_score: 34.4853)
08:30              val. loss  30.8868 (mse_score: 30.8868)
08:30    Epoch 30: train loss 34.4126 (mse_score: 34.4126)
08:30              val. loss  30.8629 (mse_score: 30.8629)
08:31    Epoch 31: train loss 34.3180 (mse_score: 34.3180)
08:31              val. loss  30.8576 (mse_score: 30.8576)
08:31    Epoch 32: train loss 34.2919 (mse_score: 34.291

...

09:02    Epoch 22: train loss 19.8801 (mse_score: 19.8801)
09:02              val. loss  34.1598 (mse_score: 34.1598)
09:02    Epoch 23: train loss 19.8040 (mse_score: 19.8040)
09:02              val. loss  34.1401 (mse_score: 34.1401)
09:03    Epoch 24: train loss 19.7144 (mse_score: 19.7144)
09:03              val. loss  34.1752 (mse_score: 34.1752)
09:03    Epoch 25: train loss 19.6179 (mse_score: 19.6179)
09:03              val. loss  34.1667 (mse_score: 34.1667)
09:04    Epoch 26: train loss 19.5295 (mse_score: 19.5295)
09:04              val. loss  34.2261 (mse_score: 34.2261)
09:04    Epoch 27: train loss 19.4490 (mse_score: 19.4490)
09:04              val. loss  34.1560 (mse_score: 34.1560)
09:04    Epoch 28: train loss 19.3671 (mse_score: 19.3671)
09:04              val. loss  34.2720 (mse_score: 34.2720)
09:05    Epoch 29: train loss 19.2803 (mse_score: 19.2803)
09:05              val. loss  34.2573 (mse_score: 34.2573)
09:05    Epoch 30: train loss 19.2050 (mse_score: 19.205

09:14              val. loss  28.0520 (mse_score: 28.0520)
09:15    Epoch 16: train loss 34.6748 (mse_score: 34.6748)
09:15              val. loss  28.1940 (mse_score: 28.1940)
09:15    Epoch 17: train loss 34.6431 (mse_score: 34.6431)
09:15              val. loss  27.9827 (mse_score: 27.9827) (*)
09:16    Epoch 18: train loss 34.5325 (mse_score: 34.5325)
09:16              val. loss  27.9971 (mse_score: 27.9971)
09:16    Epoch 19: train loss 34.4299 (mse_score: 34.4299)
09:16              val. loss  28.0854 (mse_score: 28.0854)
09:16    Epoch 20: train loss 34.3365 (mse_score: 34.3365)
09:16              val. loss  27.9781 (mse_score: 27.9781) (*)
09:17    Epoch 21: train loss 34.2600 (mse_score: 34.2600)
09:17              val. loss  28.0037 (mse_score: 28.0037)
09:17    Epoch 22: train loss 34.2231 (mse_score: 34.2231)
09:17              val. loss  28.0515 (mse_score: 28.0515)
09:18    Epoch 23: train loss 34.1220 (mse_score: 34.1220)
09:18              val. loss  28.0279 (mse_score

09:27    Epoch 8: train loss 28.3879 (mse_score: 28.3879)
09:27              val. loss  39.2800 (mse_score: 39.2800) (*)
09:27    Epoch 9: train loss 28.3546 (mse_score: 28.3546)
09:27              val. loss  39.2493 (mse_score: 39.2493) (*)
09:28    Epoch 10: train loss 28.3089 (mse_score: 28.3089)
09:28              val. loss  39.3130 (mse_score: 39.3130)
09:28    Epoch 11: train loss 28.2299 (mse_score: 28.2299)
09:28              val. loss  39.2809 (mse_score: 39.2809)
09:28    Epoch 12: train loss 28.1924 (mse_score: 28.1924)
09:28              val. loss  39.2140 (mse_score: 39.2140) (*)
09:29    Epoch 13: train loss 28.1880 (mse_score: 28.1880)
09:29              val. loss  39.2489 (mse_score: 39.2489)
09:29    Epoch 14: train loss 28.1081 (mse_score: 28.1081)
09:29              val. loss  39.3116 (mse_score: 39.3116)
09:29    Epoch 15: train loss 28.0222 (mse_score: 28.0222)
09:29              val. loss  39.2371 (mse_score: 39.2371)
09:30    Epoch 16: train loss 27.9888 (mse_sco

09:38              val. loss  20.2789 (mse_score: 20.2789) (*)
09:39    Epoch 3: train loss 37.4595 (mse_score: 37.4595)
09:39              val. loss  20.2430 (mse_score: 20.2430) (*)
09:39    Epoch 4: train loss 37.4496 (mse_score: 37.4496)
09:39              val. loss  20.2440 (mse_score: 20.2440)
09:39    Epoch 5: train loss 37.4252 (mse_score: 37.4252)
09:39              val. loss  20.2490 (mse_score: 20.2490)
09:40    Epoch 6: train loss 37.3908 (mse_score: 37.3908)
09:40              val. loss  20.2153 (mse_score: 20.2153) (*)
09:40    Epoch 7: train loss 37.3233 (mse_score: 37.3233)
09:40              val. loss  20.3200 (mse_score: 20.3200)
09:40    Epoch 8: train loss 37.2923 (mse_score: 37.2923)
09:40              val. loss  20.1848 (mse_score: 20.1848) (*)
09:41    Epoch 9: train loss 37.2333 (mse_score: 37.2333)
09:41              val. loss  20.1681 (mse_score: 20.1681) (*)
09:41    Epoch 10: train loss 37.1669 (mse_score: 37.1669)
09:41              val. loss  20.4343 (mse_

09:50  Saving input scaling information to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_5_x_means.npy and /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_5_x_stds.npy
09:50  Saving state dictionary to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_5_state_dict.pt
09:50  Saving settings to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_6_settings.json
09:50  Saving input scaling information to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_6_x_means.npy and /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_6_x_stds.npy
09:50  Saving state dictionary to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_

In [ ]:
train_ensemble('all_tight', use_tight_cuts=True)

10:11  Training 10 estimators in ensemble
10:11  Training estimator 1 / 10 in ensemble
10:11  Starting training
10:11    Method:                 sally
10:11    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local_tight/x_train_0.npy
10:11                   t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local_tight/t_xz_train_0.npy
10:11    Features:               all
10:11    Method:                 sally
10:11    Hidden layers:          (100, 100)
10:11    Activation function:    tanh
10:11    Batch size:             128
10:11    Trainer:                amsgrad
10:11    Epochs:                 50
10:11    Learning rate:          0.001 initially, decaying to 0.0001
10:11    Validation split:       0.25
10:11    Early stopping:         True
10:11    Scale inputs:           True
10:11    Regularization:         None
10:11  Loading training data
10:11  Found 1000000 samples wi

12:08    Early stopping:         True
12:08    Scale inputs:           True
12:08    Regularization:         None
12:08  Loading training data
12:08  Found 1000000 samples with 2 parameters and 27 observables
12:08  Rescaling inputs
12:08  Creating model for method sally
12:08  Training model
12:12    Epoch 5: train loss 6261.0580 (mse_score: 6261.0580)
12:12              val. loss  5305.8347 (mse_score: 5305.8347) (*)
12:15    Epoch 10: train loss 6181.0251 (mse_score: 6181.0251)
12:15              val. loss  5245.4023 (mse_score: 5245.4023) (*)
12:18    Epoch 15: train loss 6113.7660 (mse_score: 6113.7660)
12:18              val. loss  5203.8410 (mse_score: 5203.8410) (*)
12:21    Epoch 20: train loss 6056.1493 (mse_score: 6056.1493)
12:21              val. loss  5168.8501 (mse_score: 5168.8501) (*)
12:25    Epoch 25: train loss 6007.4099 (mse_score: 6007.4099)
12:25              val. loss  5147.7889 (mse_score: 5147.7889) (*)
12:28    Epoch 30: train loss 5968.0462 (mse_score: 5968.

14:11    Epoch 25: train loss 5389.5380 (mse_score: 5389.5380)
14:11              val. loss  5155.6717 (mse_score: 5155.6717) (*)
14:15    Epoch 30: train loss 5342.6621 (mse_score: 5342.6621)
14:15              val. loss  5131.3902 (mse_score: 5131.3902) (*)
14:19    Epoch 35: train loss 5306.8620 (mse_score: 5306.8620)
14:19              val. loss  5118.8099 (mse_score: 5118.8099) (*)
14:23    Epoch 40: train loss 5279.2168 (mse_score: 5279.2168)
14:23              val. loss  5105.3275 (mse_score: 5105.3275) (*)
14:26    Epoch 45: train loss 5257.6220 (mse_score: 5257.6220)
14:26              val. loss  5092.5217 (mse_score: 5092.5217) (*)
14:30    Epoch 50: train loss 5240.3143 (mse_score: 5240.3143)
14:30              val. loss  5085.7543 (mse_score: 5085.7543) (*)
14:30  Early stopping did not improve performance
14:30  Finished training
14:30  Training estimator 8 / 10 in ensemble
14:30  Starting training
14:30    Method:                 sally
14:30    Training data: x at /Users/

## Minimal observable basis

In [ ]:
train_ensemble(
    'minimal',
    use_tight_cuts=False,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

In [ ]:
train_ensemble(
    'minimal_tight',
    use_tight_cuts=True,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

In [ ]:
train_ensemble(
    'minimal_reg',
    use_tight_cuts=False,
    grad_x_regularization=0.1,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

In [ ]:
train_ensemble(
    'minimal_tight_reg',
    use_tight_cuts=True,
    grad_x_regularization=0.1,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

## Just resurrection phi

In [ ]:
train_ensemble(
    'resurrection',
    use_tight_cuts=True,
    features=[[26] for _ in range(10)]
)